In [3]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()


# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"




link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
Example 1:
Input:
[
    "https://megacorp.com/company/about",
    "https://megacorp.com/services",
    "https://megacorp.com/company/careers"
]

Output:
{
    "links": [
        {"type": "about page", "url": "https://megacorp.com/company/about"},
        {"type": "careers page", "url": "https://megacorp.com/company/careers"}
    ]
}

Example 2:
Input:
[
    "https://acmeinc.io",
    "https://acmeinc.io/our-story",
    "https://acmeinc.io/contact",
    "https://acmeinc.io/jobs-openings"
]

Output:
{
    "links": [
        {"type": "about page", "url": "https://acmeinc.io/our-story"},
        {"type": "careers page", "url": "https://acmeinc.io/jobs-openings"}
    ]
}
"""


def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt



def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)


def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result



system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt


def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))




def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)


stream_brochure("ibis acam", "https://www.ibisacam.at")

translate_system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure in German about the company for prospective German-speaking customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

def get_and_translate_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company \
     in German and respond in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

def translate_and_stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": translate_system_prompt},
            {"role": "user", "content": get_and_translate_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

translate_and_stream_brochure("Österreichische JungArbeiterBewegung", "https://www.oejab.at/")

API key looks good so far
Found links: {'links': [{'type': 'home page', 'url': 'https://www.ibisacam.at/'}, {'type': 'about page', 'url': 'https://www.ibisacam.at/vision-wir/unsere-geschichte/'}, {'type': 'careers page', 'url': 'https://www.ibisacam.at/arbeiten-bei-ibis-acam/'}, {'type': 'courses page', 'url': 'https://www.ibisacam.at/kurse/'}, {'type': 'company page', 'url': 'https://www.ibisacam.at/fuer-unternehmen/'}, {'type': 'contact page', 'url': 'https://www.ibisacam.at/kontakt/'}, {'type': 'job speed dating page', 'url': 'https://www.ibisacam.at/ausbildungjob/job-speed-dating/'}, {'type': 'training opportunities page', 'url': 'https://www.ibisacam.at/ausbildungjob/'}, {'type': 'courses overview page', 'url': 'https://www.ibisacam.at/kurse/?s='}]}


# ibis acam Brochure

---

## Welcome to ibis acam

### Who We Are
ibis acam is dedicated to empowering individuals through education and job placement services. Our mission is to transform lives and contribute to a sustainable world by offering a diverse range of courses, vocational training, and personal development programs tailored to help individuals thrive in their careers and personal lives.

---

### Our Offerings
- **Educational Programs**: Courses in professional orientation, digital training, language courses, and personal development.
- **Vocational Training**: Programs like AQUA (Job-related Qualification) help equip individuals with skills necessary for work in various fields including IT and healthcare.
- **Recruitment Services**: Customized recruitment solutions for businesses seeking skilled personnel, featuring tailored training programs.

---

### Innovation in Education
At ibis acam, we believe in digital readiness. Our initiative "Digital Überall" aims to promote digital literacy through workshops focusing on internet safety, artificial intelligence, and more, making technology accessible and practical for communities.

### Key Programs
- **Youth College**: Supporting young individuals in finding job opportunities and apprenticeships.
- **Language Courses**: Advancing language skills for better integration and participation in Austrian society.

---

### Company Culture
At ibis acam, we see ourselves as facilitators and companions in a lifelong learning journey. Our team is committed to fostering a collaborative environment filled with mutual respect and a shared mission to empower everyone we serve.

### Why Work With Us?
- **Impactful Work**: Contribute to societal change by guiding individuals toward professional success.
- **Collaborative Community**: Join a supportive team where every member plays a crucial role in achieving a collective mission.
- **Career Growth**: Opportunities for personal and professional development through continuous training and coaching.

---

### Join Our Team
Are you passionate about making a difference? Explore open positions at ibis acam and become part of a dynamic community focused on education and empowerment.

### Contact Us
Interested in learning more? Reach out to us or visit our website to explore our offerings, join our community, or advance your career with us.

---

### Connect With Us
- **Website**: [ibis acam](https://www.ibisacam.com)
- **Social Media**: Follow us to stay updated on upcoming workshops and career opportunities!

---

Thank you for considering ibis acam – together, let’s make a difference!

Found links: {'links': [{'type': 'about page', 'url': 'https://www.oejab.at/ueber-die-oejab'}, {'type': 'careers page', 'url': 'https://www.oejab.at/ueber-die-oejab/jobs'}, {'type': 'news and events', 'url': 'https://www.oejab.at/ueber-die-oejab/newsundevents'}, {'type': 'education integration', 'url': 'https://www.oejab.at/bildung-integration'}, {'type': 'senior care', 'url': 'https://www.oejab.at/seniorinnen'}]}


# ÖJAB - Österreichische Jungarbeiterbewegung

## Über uns
Die Österreichische Jungarbeiterbewegung (ÖJAB) ist ein gemeinnütziger Verein, der sich der Unterstützung von Studierenden und Jugendlichen widmet. Wir bieten nicht nur preiswerte Wohnmöglichkeiten in 23 Studenten- und Jugendheimen in ganz Österreich, sondern engagieren uns auch in den Bereichen Pflege, Bildung, Integration und Entwicklungszusammenarbeit. Mit einem umfassenden Angebot möchten wir ein Zuhause sowie Chancen für alle Menschen schaffen.

## Unsere Angebote

### Wohnen
- **Studentenwohnheime:** Preiswerte Unterkünfte in Wien, Graz, Salzburg, Eisenstadt, Krems, Mödling und Bad Gleichenberg.
- **Moderne Einrichtungen:** Unsere Heime bieten Fitnessräume, Saunen, Seminarräume und Gemeinschaftsräume, um eine angenehme Wohnatmosphäre zu schaffen.

### Pflege
- **Seniorenpflege:** Professionelle und liebevolle Pflege in unseren Wohn- und Pflegeheimen für ältere Menschen. 
- **Betreutes Wohnen:** Individuelle Lebensqualität fördern auf stationäre oder mobile Weise.

### Bildung & Integration
- **Bildungsangebote:** Berufliche Orientierungs- und Ausbildungsprogramme sowie Sprachkurse.
- **Flüchtlingshilfe:** Integration und Unterstützung von Geflüchteten, um sozialen Zusammenhalt zu stärken.

### Entwicklungszusammenarbeit
- **Internationale Projekte:** Unser Engagement in Burkina Faso zielt auf nachhaltige Entwicklung und soziale Gerechtigkeit.

## Unternehmenskultur
Die ÖJAB legt großen Wert auf ein respektvolles und inklusives Arbeitsumfeld, in dem Teamarbeit und persönliche Weiterentwicklung gefördert werden. Wir glauben an die Förderung von Talenten, Engagement in der Gemeinschaft und an innovative Ansätze zur Lösung gesellschaftlicher Herausforderungen.

## Karriere
Wir suchen engagierte Mitarbeiter, die Teil unseres Teams werden möchten. Ob im Pflegebereich, in der Bildung oder in der Verwaltung – bei uns finden Sie vielfältige Karrieremöglichkeiten.

- **Aktuelle Jobangebote:** Sie finden unser Stellenangebot auf unserer Webseite.
- **Ehrenamt:** Wir freuen uns auch über freiwillige Helfer in verschiedenen Bereichen.

Besuchen Sie uns unter [www.oejab.at](http://www.oejab.at) für mehr Informationen über unsere Angebote, Stipendien und die Möglichkeit, Teil unserer Gemeinschaft zu werden.

---

**Ihr Zuhause. Ihre Zukunft. Ihre ÖJAB.**